# Árboles de decisión

Esta variante apunta a generar una versión del ejercicio anterior usando la variable del modelo de la turbina considerando otros aspectos que se habían dejado de lado. Además, se trata de responder a la inquietud surgida respecto de la importancia que le asigna, el modelo, a las características

Dado que la variable es categórica entonces se debe transformar.

La idea es ver si tenemos resultados distintos

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
data_frame = pd.read_csv("turbines_df.csv")
data_frame.head()

,turbine_capacity,rotor_diameter_m,hub_height_m,commissioning_date,province_territory,model
0,150,23.0,30.0,1993,Alberta,Other
1,600,44.0,40.0,1997,Alberta,Other
2,600,44.0,50.0,1998,Alberta,Other
3,600,44.0,50.0,1998,Alberta,Other
4,600,44.0,50.0,1998,Alberta,Other


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_frame['model_encoder'] = le.fit_transform(data_frame.model)
data_frame

,turbine_capacity,rotor_diameter_m,hub_height_m,commissioning_date,province_territory,model,model_encoder
0,150,23.0,30.0,1993,Alberta,Other,5
1,600,44.0,40.0,1997,Alberta,Other,5
2,600,44.0,50.0,1998,Alberta,Other,5
3,600,44.0,50.0,1998,Alberta,Other,5
4,600,44.0,50.0,1998,Alberta,Other,5
...,...,...,...,...,...,...,...
6473,660,47.0,50.0,2001,Saskatchewan,Other,5
6474,660,47.0,50.0,2001,Saskatchewan,Other,5
6475,660,47.0,50.0,2001,Saskatchewan,Other,5
6476,150,23.0,30.0,1993,Other,Other,5


In [7]:
data_frame.model_encoder.unique()

array([ 5,  7,  1,  2, 10,  6,  0,  9,  4,  3,  8])

In [38]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

X = data_frame.drop(columns = ["province_territory", "model", "turbine_capacity"])
y = data_frame.turbine_capacity

# Separar los datos de "train" en entrenamiento y prueba para probar el modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 29)
modelo = DecisionTreeRegressor(max_depth = 5, random_state = 29)

# Entrenamiento del modelo
modelo.fit(X_train, y_train)

# Validación del modelo
Y_pred = modelo.predict(X_test)

# Evaluación del modelo
print('Precisión del modelo (en entrenamiento):', modelo.score(X_train, y_train))
print('Precisión del modelo (en validación):', modelo.score(X_test, y_test))
print('Coeficiente de determinación: %.2f' % r2_score(y_test, Y_pred))

print("****")
print('MAE in train:', mean_absolute_error(modelo.predict(X_train), y_train))
print('MSE in train:', np.sqrt(mean_squared_error(modelo.predict(X_train), y_train)))
print('MAE in test:', mean_absolute_error(Y_pred, y_test))
print('RMSE in test:', np.sqrt(mean_squared_error(Y_pred, y_test)))
history = [r2_score(y_test, Y_pred)]
history_mse_train = [np.sqrt(mean_squared_error(modelo.predict(X_train), y_train))]
history_mse_test = [np.sqrt(mean_squared_error(Y_pred, y_test))]

Precisión del modelo (en entrenamiento): 0.8506190234718503
Precisión del modelo (en validación): 0.8526286622769065
Coeficiente de determinación: 0.85
****
MAE in train: 120.44395918864444
MSE in train: 235.00018638191395
MAE in test: 115.60932332396071
RMSE in test: 229.0406747988052


## Importancia de las características

In [25]:
feature_importance = pd.DataFrame(modelo.feature_importances_, index=X_train.columns)
print(feature_importance.sort_values(by=0, ascending=False)[:10])

                           0
rotor_diameter_m    0.673625
commissioning_date  0.152331
model_encoder       0.150712
hub_height_m        0.023331


## Árbol de decisión SIN hiperparámetros

In [39]:
modelo_shp = DecisionTreeRegressor(random_state = 29)

# Entrenamiento del modelo
modelo_shp.fit(X_train, y_train)

# Validación del modelo
Y_pred = modelo_shp.predict(X_test)

# Evaluación del modelo
print('Precisión del modelo (en entrenamiento):', modelo_shp.score(X_train, y_train))
print('Precisión del modelo (en validación):', modelo_shp.score(X_test, y_test))
print('Coeficiente de determinación: %.2f' % r2_score(y_test, Y_pred))

print("****")
print('MAE in train:', mean_absolute_error(modelo_shp.predict(X_train), y_train))
print('MSE in train:', np.sqrt(mean_squared_error(modelo_shp.predict(X_train), y_train)))
print('MAE in test:', mean_absolute_error(Y_pred, y_test))
print('MSE in test:', np.sqrt(mean_squared_error(Y_pred, y_test)))

history += [r2_score(y_test, Y_pred)]
history_mse_train += [np.sqrt(mean_squared_error(modelo_shp.predict(X_train), y_train))]
history_mse_test += [np.sqrt(mean_squared_error(Y_pred, y_test))]

Precisión del modelo (en entrenamiento): 0.9899472426707983
Precisión del modelo (en validación): 0.9896659838074617
Coeficiente de determinación: 0.99
****
MAE in train: 14.893067415153421
MSE in train: 60.96255481501382
MAE in test: 14.711119391453472
MSE in test: 60.65137772558558


### Importancia de las características

In [27]:
eature_importance = pd.DataFrame(modelo_shp.feature_importances_, index=X_train.columns)
print(feature_importance.sort_values(by=0, ascending=False)[:10])

                           0
rotor_diameter_m    0.673625
commissioning_date  0.152331
model_encoder       0.150712
hub_height_m        0.023331


## Árbol de decisión usando hiper parámetros

En scikit-learn podemos usar varios hiper-parámetros para configurar la forma en que se regularizan los árboles de decisión. 

Los más usados:

+ max_depth: la profundidad máxima del árbol. 
+ min_samples_split: número mínimo de muestras necesarias antes de dividir este nodo. También se puede expresar en porcentaje.
+ min_samples_leaf: número mínimo de muestras que debe haber en un nodo final (hoja). También se puede expresar en porcentaje.
+ max_leaf_nodes: número máximo de nodos finales

In [40]:
modelo_hp = DecisionTreeRegressor(min_samples_split=24, min_samples_leaf=10, max_leaf_nodes=300, 
                               random_state = 29)

# Entrenamiento del modelo
modelo_hp.fit(X_train, y_train)

# Validación del modelo
Y_pred = modelo_hp.predict(X_test)

# Evaluación del modelo
print('Precisión del modelo (en entrenamiento):', modelo_hp.score(X_train, y_train))
print('Precisión del modelo (en validación):', modelo_hp.score(X_test, y_test))
print('Coeficiente de determinación: %.2f' % r2_score(y_test, Y_pred))
print("****")
print('MAE in train:', mean_absolute_error(modelo_hp.predict(X_train), y_train))
print('MSE in train:', np.sqrt(mean_squared_error(modelo_hp.predict(X_train), y_train)))
print('MAE in test:', mean_absolute_error(Y_pred, y_test))
print('RMSE in test:', np.sqrt(mean_squared_error(Y_pred, y_test)))

history += [r2_score(y_test, Y_pred)]
history_mse_train += [np.sqrt(mean_squared_error(modelo_hp.predict(X_train), y_train))]
history_mse_test += [np.sqrt(mean_squared_error(Y_pred, y_test))]

Precisión del modelo (en entrenamiento): 0.9837217948986271
Precisión del modelo (en validación): 0.9842337842598526
Coeficiente de determinación: 0.98
****
MAE in train: 22.87489781856189
MSE in train: 77.57536322522965
MAE in test: 21.62255730081868
RMSE in test: 74.9151811794363


## Importancia de las caraterísticas

In [29]:
feature_importance = pd.DataFrame(modelo_hp.feature_importances_, index=X_train.columns)
print(feature_importance.sort_values(by=0, ascending=False)[:10])

                           0
rotor_diameter_m    0.677602
commissioning_date  0.165483
model_encoder       0.133646
hub_height_m        0.023269


In [51]:
# feature_importances_ es un vector con la importancia estimada de cada atributo
# Otra forma

for name, importance in zip(X_train.columns, modelo_hp.feature_importances_):
    print(name + ': ' + str(importance))

rotor_diameter_m: 0.6776018067360409
hub_height_m: 0.02326864827448778
commissioning_date: 0.1654834936389406
model_encoder: 0.13364605135053076


## Resumen de resultados

In [45]:
df=pd.DataFrame({'tipo': ['Un HP', 'Sin HP', 'Con n HP'], 'r2': history, 'mse_train' : history_mse_train, 'mse_test' : history_mse_test })
df.columns = ['Configuración', 'R2', 'MSE TRAIN', 'MSE TEST'] 
df.sort_values(by = 'R2', ascending = False)

,Configuración,R2,MSE TRAIN,MSE TEST
1,Sin HP,0.989666,60.962555,60.651378
2,Con n HP,0.984234,77.575363,74.915181
0,Un HP,0.852629,235.000186,229.040675
